# Data to Pandas table

**Purpose of script:**

Put mw and opt data to one pandas dataframe, create features. One row per pixel.

- In: opt and mw data, all files
- Out: one file (maybe several due to size constraints) (csv or parquet) with table of features prepared to be used in model

In [1]:
# import rioxarray
import xarray
# import rasterio

# from os import listdir
# from os.path import isfile, join

import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm

Relevant paths:

In [2]:
mw_path = r"../Data/microwave-rs/mw_interpolated/2019-07-01_mw.tif"
opt_path = r"../Data/optical-rs/2019-07-01_grain_diameter.tif"

In [3]:
out_path = r"../Data/combined/"

Load data:

In [4]:
# load all files
# put to one xarray?

In [5]:
#TEMP Data load:
data_opt = xarray.open_dataarray(opt_path)
data_mw = xarray.open_dataarray(mw_path)

To pandas dataframe:

In [6]:
# for loop!

# convert mw to pandas
df_mw = data_mw.to_dataframe()
df_opt = data_opt.to_dataframe()
# fix index
df_mw = df_mw.reset_index()
# remove columns: spacial_ref, band
df_mw = df_mw[['x', 'y', 'band_data']]
# rename
df_mw.rename({'band_data': 'mw_value'}, axis=1, inplace=True)
# ----------------------
# convert opt to pandas
df_opt = data_opt.to_dataframe()
# fix index
df_opt = df_opt.reset_index()
# remove columns: spacial_ref, band
df_opt = df_opt[['x', 'y', 'band_data']]
# rename
df_opt.rename({'band_data': 'opt_value'}, axis=1, inplace=True)
# ----------------------
# merge
df_comb = pd.merge(df_mw, df_opt, how = 'left', on = ['y', 'x']) # left smaller mw, right - opt
# fill na for masked opt data
df_comb.opt_value.fillna(-1, inplace=True)

In [ ]:
# baptiste:
# open_mfdataset() 
# concat dim time (to put to ine file?)

Neighbor dataframe

In [ ]:
def get_neighbors(mat, a, b):
    neighbors = [mat[i][j] if (i > -1 and j > -1 and j < len(mat[0]) and i < len(mat)) else np.nan for i in range(a-1, a+2) for j in range(b-1, b+2) ]
    return neighbors

In [8]:
index_list = [(i,j) for i in range(data_opt.shape[1]) for j in range(data_opt.shape[2])]
value_list = []
data = data_opt.values[0]

for i in tqdm(index_list):
    neighbor = get_neighbors(data, *i)
    #print([i[0], i[1]])
    neighbor += [i[0], i[1]]
    value_list.append(neighbor)

  0%|          | 0/3995569 [00:00<?, ?it/s]


NameError: name 'get_neighbors' is not defined

In [ ]:
df_neighbors = pd.DataFrame(value_list, columns = ['v1', 'v2', 'v3','v4', 'v5', 'v6','v7', 'v8', 'v9', 'x', 'y'])

Merge neighbors

In [ ]:
df_comb1 = pd.merge(df_comb, df_neighbors, how = 'left', on = ['y', 'x'])


: 

: 

In [ ]:
df_comb1

Write to csv/parquet:

In [12]:
# write to csv
#df_comb.to_csv(out_path + 'melt_2019-07-01.csv', index= False)
df_comb.to_parquet(out_path + 'melt_2019-07-01.parquet.gzip', index= False)